# Submission

In [1]:
# Magic to automatically update imports if functions in utils are changed
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import seaborn as sns
import sklearn
import matplotlib.pyplot as plt
from tqdm import tqdm
from pathlib import Path

In [2]:
from sklearn.model_selection import train_test_split

stores_total_train = pd.read_csv("data/stores_train.csv")
stores_test = pd.read_csv("data/stores_test.csv")
stores_extra = pd.read_csv("data/stores_extra.csv")
stores_train, stores_val = train_test_split(stores_total_train, test_size=0.2, random_state=0)

In [3]:
from utils import split_plaace_cat

stores_train = split_plaace_cat(stores_train)
stores_val = split_plaace_cat(stores_val)
stores_extra = split_plaace_cat(stores_extra)
stores_test = split_plaace_cat(stores_test)



store_dataframes = {
    "train": stores_train, 
    "val": stores_val, 
    "extra": stores_extra, 
    "test": stores_test, 
    }

In [7]:
from utils import create_geographical_columns, create_chain_and_mall_columns, generate_rev_dict, generate_chain_rev_dict, create_mean_chain_rev_col

chain_count = stores_train["chain_name"].value_counts().to_dict()
lower_limit = 10
rev_dict, mean_revenue = generate_rev_dict(stores_train)

for df_name, df in tqdm(store_dataframes.items()):
    df = create_geographical_columns(df)
    df = create_chain_and_mall_columns(df, chain_count, lower_limit=lower_limit)

chain_rev_dict = generate_chain_rev_dict(store_dataframes["train"])
for df_name, df in tqdm(store_dataframes.items()):
    df = create_mean_chain_rev_col(df, bounded_chain_revs=chain_rev_dict)


100%|██████████| 4/4 [00:00<00:00, 507.94it/s]


In [8]:
from utils import concat_df_keep_unq_index
concat_df = concat_df_keep_unq_index(stores_train, stores_extra)

!NB next cell takes 3 minutes to run (if comp_plaace_gran. = [1, 2, 3, 4]) (on M1 Mac with 16GB RAM)

(outdated) If you have the .csv files temp_data/closest_comp_\{df_name\}, skip running this cell and run the cell below it instead

In [10]:
from utils import find_dist_to_nearest_comp

nearest_comp_plaace_cat_gran = [1, 2, 3, 4]
n_nearest_comp = [1, 2, 3, 4, 5, 7, 10]

store_dataframes["train"] = find_dist_to_nearest_comp(
    store_dataframes["train"], 
    nearest_comp_plaace_cat_gran, 
    n_nearest_comp, 
    training=True, 
    training_df=concat_df,
    )

In [11]:
store_dataframes["val"] = find_dist_to_nearest_comp(
    store_dataframes["val"], 
    nearest_comp_plaace_cat_gran, 
    n_nearest_comp, 
    training=True, 
    training_df=concat_df,
    )

In [12]:
store_dataframes["test"] = find_dist_to_nearest_comp(
    store_dataframes["test"], 
    nearest_comp_plaace_cat_gran, 
    n_nearest_comp, 
    training=False,
    training_df=concat_df
)

In [13]:
comp_plaace_cols = list(store_dataframes["train"].columns)[-((len(nearest_comp_plaace_cat_gran) + len(n_nearest_comp))*5+1):]

In [ ]:
for df_name, df in store_dataframes.items():
    merge_df = pd.read_csv(f"temp_data/closest_comp_{df_name}.csv")
    store_dataframes[df_name] = df.merge(merge_df, left_on="store_id", right_on="store_id")


In [14]:
dist_dict = store_dataframes["train"][['revenue'] + comp_plaace_cols].corr().iloc[0].to_dict()
sorted_relevant_dist_cols = [[k, v] for k, v in sorted(dist_dict.items(), key=lambda item: abs(item[1]), reverse=True)]
relevant_cols = sorted_relevant_dist_cols[1:7]
relevant_cols = [r[0] for r in relevant_cols]

!NB next cell takes 15 minutes to run. Grab a coffee or somtething while you wait :) (on M1 Mac with 16GB RAM)

If you have the .csv files temp_data/closest_bus_stop_\{df_name\}, skip running this cell and run the cell below it instead

In [15]:
from bus_utils import find_closest_bus_stop

bus_stop_n = [1, 3, 5, 7]
bus_mean = True
bus_sum = True

bus_stop_columns = []

if(bus_sum):
    bus_stop_columns += [f"closest_bus_stop_sum_{i}" for i in bus_stop_n]

if(bus_mean):
    bus_stop_columns += [f"closest_bus_stop_mean_{i}" for i in bus_stop_n]


for df_name, df in tqdm(store_dataframes.items()):
    store_dataframes[df_name] = find_closest_bus_stop(df, bus_stop_n, _sum=bus_sum, _mean=bus_mean)    
    filepath = Path(f"temp_data/closest_bus_stop_{df_name}.csv")  
    filepath.parent.mkdir(parents=True, exist_ok=True)  
    df.to_csv(filepath, columns=["store_id"] + bus_stop_columns, index=True)


  0%|          | 0/4 [00:04<?, ?it/s]


KeyboardInterrupt: 

In [21]:
for df_name, df in store_dataframes.items():
    merge_df = pd.read_csv(f"temp_data/closest_bus_stop_{df_name}.csv")
    bus_stop_columns = list(merge_df.columns)
    store_dataframes[df_name] = df.merge(merge_df, left_on="store_id", right_on="store_id")

In [22]:
bus_dict = store_dataframes["train"][['revenue'] + bus_stop_columns].corr().iloc[0].to_dict()
bus_sorted_relevant_dist_cols = [[k, v] for k, v in sorted(dist_dict.items(), key=lambda item: abs(item[1]), reverse=True)]
bus_relevant_cols = sorted_relevant_dist_cols[1:7]
bus_relevant_cols = [r[0] for r in relevant_cols]

In [23]:
bus_dict

{'revenue': 1.0,
 'closest_bus_stop_sum_1': -0.022249244863896008,
 'closest_bus_stop_sum_3': -0.02770996962488185,
 'closest_bus_stop_sum_5': -0.029157619550420597,
 'closest_bus_stop_sum_7': -0.03074876819639974,
 'closest_bus_stop_mean_1': -0.022249244863896008,
 'closest_bus_stop_mean_3': -0.02770996962488174,
 'closest_bus_stop_mean_5': -0.029157619550420573,
 'closest_bus_stop_mean_7': -0.03074876819639974}

In [24]:
from grunnkrets import make_grunnkrets_df

full_population_dataframes = {}

for df_name, df in tqdm(store_dataframes.items()):
    full_population_dataframes[df_name] = make_grunnkrets_df(df)

 25%|██▌       | 1/4 [00:01<00:03,  1.21s/it]/Users/oskarjor/NTNU/2022FALL/TDT4173/TDT4173/grunnkrets.py:101: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  full_population_df[f'{level}.income_density_log']  = np.log1p(full_population_df[f'{level}.income_density'])
/Users/oskarjor/NTNU/2022FALL/TDT4173/TDT4173/grunnkrets.py:104: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  full_population_df[f'{level}.pop_density'] = full_population_df[f'{level}.tot_pop']/full_population_df[f'{level}.area_km2']
/Users/oskarjor/NTNU/2022FALL/TD

In [25]:
for df_name, df in store_dataframes.items():
    store_dataframes[df_name] = df.merge(
        full_population_dataframes[df_name], 
        left_index=True,
        right_index=True,
        how="outer", 
        suffixes=('', '_redundant')
    )
    store_dataframes[df_name].drop(store_dataframes[df_name].filter(regex='_redundant$').columns, axis=1, inplace=True)

In [26]:
fylke_relevant_features = [col_name for col_name in store_dataframes["train"].columns if col_name.startswith("fylke.")]
kommune_relevant_features = [col_name for col_name in store_dataframes["train"].columns if col_name.startswith("kommune.")]
delomrade_relevant_features = [col_name for col_name in store_dataframes["train"].columns if col_name.startswith("delomrade.")]
grunnkrets_relevant_features = [col_name for col_name in store_dataframes["train"].columns if col_name.startswith("grunnkrets_id.")]

In [27]:
from utils import mean_func_rev, generate_rev_dict

for i in range(1, 5):
    rev_dict, mean_revenue = generate_rev_dict(stores_train, i)
    for df_name, df in store_dataframes.items():
        df["mean_revenue_" + str(i)] = df["plaace_cat_" + str(i)].apply(lambda x: mean_func_rev(x, rev_dict, mean_revenue))
        df["log_mean_revenue_" + str(i)] = df["mean_revenue_" + str(i)].apply(lambda x: np.log1p(x))

# Training the model

In [32]:
from RMSLE import rmsle
from pred_var_utils import log1p_transform_pred_var, reverse_log1p_transform_pred_var

In [48]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder


OE_categorical_features = ["bounded_chain_name", "kommune", "delomrade", "is_grocery", "plaace_cat_3", "plaace_cat_4"]
OE_categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(missing_values=np.nan, strategy="constant")),
        ("encoder", OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)),
    ]
)

OH_categorical_features = ["fylke", "plaace_cat_2"]
OH_categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(missing_values=np.nan, strategy="constant")),
        ("encoder", OneHotEncoder(handle_unknown="ignore")),
    ]
)


numerical_features = ["lat", "lon", 
#"mean_revenue_1", "mean_revenue_2", "mean_revenue_3", "mean_revenue_4", 
"log_mean_revenue_1", "log_mean_revenue_2", "log_mean_revenue_3", "log_mean_revenue_4", 
"chain_mean_revenue"
] + delomrade_relevant_features + relevant_cols + bus_stop_columns[1:]
numerical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="mean")), 
        ("scaler", StandardScaler(with_mean=True, with_std=True))]
)


preprocessor = ColumnTransformer(
    transformers=[
        ("oe_cat", OE_categorical_transformer, OE_categorical_features),
        ("oh_cat", OH_categorical_transformer, OH_categorical_features),
        ("num", numerical_transformer, numerical_features),
    ],
    remainder='drop'
)


X_train = preprocessor.fit_transform(store_dataframes["train"])
X_val = preprocessor.transform(store_dataframes["val"])

In [49]:
store_dataframes["train"]["log_revenue"] = store_dataframes["train"].revenue.apply(lambda x: np.log1p(x))
store_dataframes["val"]["log_revenue"] = store_dataframes["val"].revenue.apply(lambda x: np.log1p(x))

In [50]:
y_train = np.array(store_dataframes["train"].log_revenue)
y_val = np.array(store_dataframes["val"].revenue)
mean_y = y_train.mean()
std_y = y_train.std()

y_train -= mean_y
y_train /= std_y

## Random Forest

In [51]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state=0, n_jobs=-1, n_estimators=250, max_features=8, min_samples_leaf=2, min_samples_split=16)

In [52]:
rf.fit(X_train, y_train)

RandomForestRegressor(max_features=8, min_samples_leaf=2, min_samples_split=16,
                      n_estimators=250, n_jobs=-1, random_state=0)

In [53]:
rf_y_pred = reverse_log1p_transform_pred_var(rf.predict(X_val), std_y=std_y, mean_y=mean_y)


In [54]:
rmsle(y_val, rf_y_pred)

0.7490999219008018

## Linear Regression

In [55]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression(n_jobs=-1)

In [56]:
lr.fit(X_train, y_train)

LinearRegression(n_jobs=-1)

In [57]:
lr_y_pred = reverse_log1p_transform_pred_var(lr.predict(X_val), std_y, mean_y)
lr_y_pred = np.array([max(0, xi) for xi in lr_y_pred])

In [58]:
rmsle(y_val, lr_y_pred)

0.7641573067553269

## Light GBM

In [59]:
from lightgbm import LGBMRegressor

lgbm = LGBMRegressor(random_state=0, n_jobs=-1, learning_rate=0.1, n_estimators=100, reg_lambda=0.01)

In [60]:
lgbm.fit(X_train, y_train)

LGBMRegressor(random_state=0, reg_lambda=0.01)

In [61]:
lgbm_y_pred = reverse_log1p_transform_pred_var(lgbm.predict(X_val), std_y, mean_y)
lgbm_y_pred = np.array([max(0, xi) for xi in lgbm_y_pred])

In [62]:
rmsle(y_val, lgbm_y_pred)

0.7388650180784758

## CatBoost

In [63]:
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV

cb = CatBoostRegressor(
    random_seed=0, 
    verbose=False, 
    depth= 4, l2_leaf_reg= 5, learning_rate= 0.1
    )

cb_params = grid = {
    'learning_rate': [0.05, 0.1, 0.3],
    'depth': [2, 3, 4, 5, 6],
    'l2_leaf_reg': [4, 5], 
    
    }

#cb_clf = cb.grid_search(cb_params, X=X_train, y=y_train)

In [ ]:
cb_clf["params"]

In [ ]:
cb_clf["params"]

In [64]:
cb.fit(X_train, y_train)

In [65]:
cb_y_pred = reverse_log1p_transform_pred_var(cb.predict(X_val), std_y, mean_y)
cb_y_pred = np.array([max(0, xi) for xi in cb_y_pred])

In [66]:
rmsle(y_val, cb_y_pred)

0.7364148294506386

## Stacking classifiers

In [67]:
from sklearn.ensemble import StackingRegressor

estimators = [
    ('rf', rf), 
    ('lf', lr), 
    ('lgbm', lgbm), 
]

reg = StackingRegressor(
    estimators=estimators,
    final_estimator=RandomForestRegressor(n_estimators=200, random_state=0, n_jobs=-1)
)

In [68]:
reg.fit(X_train, y_train)

StackingRegressor(estimators=[('rf',
                               RandomForestRegressor(max_features=8,
                                                     min_samples_leaf=2,
                                                     min_samples_split=16,
                                                     n_estimators=250,
                                                     n_jobs=-1,
                                                     random_state=0)),
                              ('lf', LinearRegression(n_jobs=-1)),
                              ('lgbm',
                               LGBMRegressor(random_state=0, reg_lambda=0.01))],
                  final_estimator=RandomForestRegressor(n_estimators=200,
                                                        n_jobs=-1,
                                                        random_state=0))

In [69]:
reg_y_pred = reverse_log1p_transform_pred_var(reg.predict(X_val), std_y, mean_y)

In [70]:
rmsle(y_pred=reg_y_pred, y_true=y_val)

0.7666382490289607

## Creating the submission

### Create pred with only store_train

In [ ]:
# Predict on the test set 
X_test = preprocessor.transform(store_dataframes["test"])
y_test_pred = reverse_log1p_transform_pred_var(lgbm.predict(X_test), std_y, mean_y)

# Generate submission dataframe 
# NOTE: It is important that the ID and predicted values match
submission = pd.DataFrame()
submission['id'] = stores_test.store_id 
submission['predicted'] = np.asarray(y_test_pred)

# Save it to disk (`index=False` means don't save the index in the csv)
submission.to_csv('submission.csv', index=False)

### Create pred with both store_train and store_val

In [71]:
# Predict on the test set 
total_train_df = concat_df_keep_unq_index(store_dataframes["train"], store_dataframes["val"])

y_train = np.array(total_train_df.log_revenue)
mean_y = y_train.mean()
std_y = y_train.std()

y_train -= mean_y
y_train /= std_y

X_train = preprocessor.fit_transform(total_train_df)
X_test = preprocessor.transform(store_dataframes["test"])
cb.fit(X_train, y_train)
y_test_pred = reverse_log1p_transform_pred_var(cb.predict(X_test), std_y, mean_y)

# Generate submission dataframe 
# NOTE: It is important that the ID and predicted values match
submission = pd.DataFrame()
submission['id'] = stores_test.store_id 
submission['predicted'] = np.asarray(y_test_pred)

# Save it to disk (`index=False` means don't save the index in the csv)
submission.to_csv('submission.csv', index=False)